
# MRSA Modeling – Clean Evaluation Notebook
This notebook provides a comprehensive evaluation of multiple machine learning models for predicting MRSA infection rates across U.S. counties and hospital types.

Section 1: Data Loading + Cleaning + Helper Functions
- 1.1 Package Imports
- 1.2 Load Data
- 1.3 Create datasets for Base(no SDOH), SDOH, and SDOH + Lag features
- 1.4 Helper fuctions
    - 1.4.1 Model parameters
    - 1.4.2 Modeling pipeline

Section 2: Initial Model Testing
- Section 2.1: Model testing
    - 2.1.1 Linear Regression - without SDOH
    - 2.1.2 Linear Regression - with SDOH
    - 2.1.3 ARIMA - without SDOH
    - 2.1.4 Random Forrest (RF) Regressor without SDOH
    - 2.1.5 RF Regressor with SDOH
    - 2.1.6 RF Regressor with SDOH + lag features
    - 2.1.7 XGBoost Regressor with SDOH + lag features
    - 2.1.8 Deep Learning (Keras)
- Section 2.2: Model Result Compilation

Section 3: Hyperparameter Tuning
- RandomizedSearchCV to optimize Random Forest + SDOH + lag features
- Identify best parameters for production modeling

Section 4: Validation & Interpretation
- Rolling-horizon cross-validation (5 folds, 2017–2024)
- Feature importance analysis to identify key drivers of infection burden

Key Findings
- Best model: Random Forest with SDOH + lag features (tuned)
- Top predictors: Labor Force, patient-days in major teaching hospitals, ranked population
- Top 10 features: Account for ~78% of total model importance
- Temporal signal: Lagged infections provide meaningful predictive value

Data Sources
- Input: `src/cse_8803_epi_project/data/mrsa_data/clean_data/county_year_mrsa.csv` (aggregated facility-level infections by county/year) + `src/cse_8803_epi_project/data/sdoh_data/clean_data/combined_measure_data.csv` (SDOH)
- Time period: 2017–2024
- Geographic unit: California counties
- Hospital types: Critical Access, Long-Term Acute Care, Major Teaching, Pediatric, Rehabilitation, Community (small/medium/large)


# Section 1: Data Loading + Cleaning + Helper Functions

## 1.1 Package Imports

In [3]:
import numpy as np
import pandas as pd

from sklearn.base import BaseEstimator, RegressorMixin
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

from tensorflow import keras
from tensorflow.keras import layers

from typing import Callable, Dict, List, Tuple

from xgboost import XGBRegressor


## 1.2 Load Data

In [5]:
county_year_df = pd.read_csv(rf'../../data/mrsa_data/clean_data/county_year_mrsa.csv')
county_year_df = county_year_df.sort_values(by=['County', 'Year'])
county_year_df = county_year_df.drop(
    columns=county_year_df.filter(like="infections_predicted").columns
)

df = county_year_df.copy()
sdoh_df = pd.read_csv(rf'../../data/sdoh_data/clean_data/combined_measure_data.csv')


hospital_types = [
    "Critical Access",
    "Free-Standing Rehabilitation Hospital",
    "Long-Term Acute Care",
    "Major Teaching",
    "Pediatric",
    "Rehabilitation Unit",
    "cl",
    "cm",
    "cs"
]

# infection & patient-day column names
infection_cols = {ht: f"infections_reported__{ht}" for ht in hospital_types}
patient_cols   = {ht: f"patient_days__{ht}"        for ht in hospital_types}
patient_day_cols = [c for c in df.columns if c.startswith("patient_days__")]
target_total_col = "Infections_Reported_Total"

# SDOH columns 
sdoh_cols = [
    c for c in sdoh_df.columns
    if c not in ["FIPS", "Year", "State", "County"]
]


## 1.3 Create datasets for Base(no SDOH), SDOH, and SDOH + Lag features

In [ ]:
# base df: facility-level county-year
df_base = df.copy()

# sdoh df: left join
df_sdoh = df.merge(sdoh_df, on=["Year", "State", "County"], how="left")

# sdoh + lag df:
df_sdoh_lag = df_sdoh.sort_values(["State", "County", "Year"]).copy()

for ht, inf_col in infection_cols.items():
    lag_col = f"{inf_col}_lag1"
    df_sdoh_lag[lag_col] = (
        df_sdoh_lag
        .groupby(["State", "County"])[inf_col]
        .shift(1)
    )
# drop rows with missing lagged infection values (first year per county)
df_sdoh_lag = df_sdoh_lag.dropna(
    subset=[f"{infection_cols[ht]}_lag1" for ht in hospital_types]
)

In [8]:
print("df_base:", df_base.shape)
print("df_sdoh:", df_sdoh.shape)
print("df_sdoh_lag:", df_sdoh_lag.shape)


df_base: (446, 24)
df_sdoh: (446, 55)
df_sdoh_lag: (390, 64)


## 1.4 Helper Functions

### 1.4.1 Model Parameters

In [11]:
def build_rf():
    return RandomForestRegressor(
        n_estimators=500,
        max_depth=None,
        random_state=42,
        n_jobs=-1
    )

def build_xgb():
    return XGBRegressor(
        n_estimators=500,
        max_depth=6,
        learning_rate=0.05,
        subsample=0.9,
        colsample_bytree=0.9,
        objective="reg:squarederror",
        random_state=42,
        n_jobs=-1,
        tree_method="hist"
    )

### 1.4.2 Modeling Pipeline

In [10]:

#  build preprocessor (one-hot for State & County)
def make_preprocessor(feature_cols, cat_cols=("State", "County")):
    """Build a ColumnTransformer that one-hot encodes categorical columns and passes through the rest."""
    cat_cols = [c for c in cat_cols if c in feature_cols]
    transformer = ColumnTransformer(
        transformers=[
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols)
        ],
        remainder="passthrough"
    )
    return transformer


# evaluator function
def fit_and_evaluate_config(
    config_name: str,
    data: pd.DataFrame,
    feature_cols: list,
    model_builder: Callable[[], object],
    hospital_types: list,
    infection_cols: dict,
    target_total_col: str,
    train_year_start: int = 2017,
    train_year_end: int = 2023,
    test_year: int = 2024,
) -> Tuple[pd.DataFrame, pd.DataFrame]:

    # Filter years
    df_train = data[(data["Year"] >= train_year_start) & (data["Year"] <= train_year_end)].copy()
    df_test  = data[data["Year"] == test_year].copy()

    # ---------- Overall model (total infections) ----------
    # Only require the target to be non-null (features have been imputed already)
    df_train_overall = df_train.dropna(subset=[target_total_col])
    df_test_overall  = df_test.dropna(subset=[target_total_col])

    overall_metrics_df = pd.DataFrame([{
        "Config": config_name,
        "MAE_2024": np.nan,
        "MSE_2024": np.nan,
        "R2_2024": np.nan
    }])

    if not df_test_overall.empty and not df_train_overall.empty:
        X_train_overall = df_train_overall[feature_cols]
        X_test_overall  = df_test_overall[feature_cols]

        preprocessor = make_preprocessor(feature_cols)
        base_model = model_builder()

        overall_pipe = Pipeline(steps=[
            ("preprocess", preprocessor),
            ("model", base_model)
        ])

        y_train_total = df_train_overall[target_total_col]
        y_test_total  = df_test_overall[target_total_col]

        overall_pipe.fit(X_train_overall, y_train_total)
        y_pred_total = overall_pipe.predict(X_test_overall)

        overall_mae = mean_absolute_error(y_test_total, y_pred_total)
        overall_mse = mean_squared_error(y_test_total, y_pred_total)
        overall_r2  = r2_score(y_test_total, y_pred_total)

        overall_metrics_df.loc[0, "MAE_2024"] = overall_mae
        overall_metrics_df.loc[0, "RMSE_2024"] = np.sqrt(overall_mse)
        overall_metrics_df.loc[0, "R2_2024"]  = overall_r2

    # ---------- Per-facility models ----------
    facility_rows = []

    for ht in hospital_types:
        tgt_col = infection_cols[ht]

        df_train_fac = df_train.dropna(subset=[tgt_col])
        df_test_fac  = df_test.dropna(subset=[tgt_col])

        if df_train_fac.empty or df_test_fac.empty:
            facility_rows.append({
                "Config": config_name,
                "Hospital_Type": ht,
                "MAE_2024": np.nan,
                "RMSE_2024": np.nan,
                "R2_2024": np.nan
            })
            continue

        X_train_fac = df_train_fac[feature_cols]
        X_test_fac  = df_test_fac[feature_cols]

        preprocessor = make_preprocessor(feature_cols)
        fac_model = model_builder()
        fac_pipe = Pipeline(steps=[
            ("preprocess", preprocessor),
            ("model", fac_model)
        ])

        y_train_fac = df_train_fac[tgt_col]
        y_test_fac  = df_test_fac[tgt_col]

        fac_pipe.fit(X_train_fac, y_train_fac)
        y_pred_fac = fac_pipe.predict(X_test_fac)

        mae = mean_absolute_error(y_test_fac, y_pred_fac)
        mse = mean_squared_error(y_test_fac, y_pred_fac)
        r2  = r2_score(y_test_fac, y_pred_fac)

        facility_rows.append({
            "Config": config_name,
            "Hospital_Type": ht,
            "MAE_2024": mae,
            "RMSE_2024": np.sqrt(mse),
            "R2_2024": r2
        })

    per_facility_df = pd.DataFrame(facility_rows)

    return overall_metrics_df, per_facility_df


Section 2: Initial Model Testing
- Section 2.1: Model testing
    - 2.1.1 Linear Regression - without SDOH
    - 2.1.2 Linear Regression - with SDOH
    - 2.1.3 ARIMA - with SDOH
    - 2.1.4 Random Forrest (RF) Regressor without SDOH
    - 2.1.5 RF Regressor with SDOH
    - 2.1.6 RF Regressor with SDOH + lag features
    - 2.1.7 XGBoost Regressor with SDOH + lag features
    - 2.1.8 Deep Learning (Keras)
- Section 2.2: Model Result Compilation

# Section 2: Initial Model Testing

## 2.1 Model Testing

In [ ]:

#basic (no sdoh, no lag)
feature_cols_rf_basic = (
    ["Year", "State", "County"] +
    patient_day_cols
)

#with sdoh
feature_cols_rf_sdoh = (
    ["Year", "State", "County"] +
    patient_day_cols +
    sdoh_cols
)

#with sdoh and lag
lag_cols = [f"{infection_cols[ht]}_lag1" for ht in hospital_types]
feature_cols_rf_lag = (
    ["Year", "State", "County"] +
    patient_day_cols +
    sdoh_cols +
    lag_cols
)

### 2.1.1 Linear Regression - without SDOH

In [ ]:

def build_lr():
    """
    Drop-in Linear Regression builder that handles NaNs
    without modifying your existing preprocessing pipeline.
    """
    return Pipeline([
        ("imputer", SimpleImputer(strategy="mean")),
        ("lr", LinearRegression())
    ])

lr_overall_no_sdoh, lr_facilities_no_sdoh = fit_and_evaluate_config(
    config_name="lr_no_sdoh",
    data=df_base,
    feature_cols=feature_cols_rf_basic,
    model_builder=build_lr,
    hospital_types=hospital_types,
    infection_cols=infection_cols,
    target_total_col=target_total_col
)


### 2.1.2 Linear Regression - with SDOH

In [54]:
lr_overall_with_sdoh, lr_facilities_with_sdoh = fit_and_evaluate_config(
    config_name="lr_with_sdoh",
    data=df_sdoh,
    feature_cols=feature_cols_rf_sdoh,
    model_builder=build_lr,
    hospital_types=hospital_types,
    infection_cols=infection_cols,
    target_total_col=target_total_col
)


### 2.1.3 ARIMA - with SDOH

In [ ]:
def _arima_metrics_for_target(
    data: pd.DataFrame,
    target_col: str,
    train_year_start: int,
    train_year_end: int,
    test_year: int,
    arima_order: Tuple[int, int, int] = (1, 1, 0),
) -> Dict[str, float]:
    df = data.copy()
    df = df.dropna(subset=[target_col])

    #  keep years up to test_year
    df = df[(df["Year"] >= train_year_start) & (df["Year"] <= test_year)]

    preds = []
    trues = []

    for county, df_c in df.groupby("County"):
        df_c = df_c.sort_values("Year")

        df_train = df_c[
            (df_c["Year"] >= train_year_start) &
            (df_c["Year"] <= train_year_end)
        ]
        df_test = df_c[df_c["Year"] == test_year]


        if df_train.empty or df_test.empty:
            continue

        y_train = df_train[target_col].values.astype(float)
        y_test = df_test[target_col].values.astype(float)
        if len(y_train) < 3:
            continue

        try:
            model = ARIMA(y_train, order=arima_order)
            fitted = model.fit()
            y_forecast = fitted.forecast(steps=1)[0]
        except Exception:
            continue

        preds.append(y_forecast)
        trues.append(y_test[0])

    if len(trues) == 0:
        return {"MAE_2024": np.nan, "RMSE_2024": np.nan, "R2_2024": np.nan}

    y_true = np.array(trues)
    y_pred = np.array(preds)

    mae = mean_absolute_error(y_true, y_pred)
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)

    return {"MAE_2024": mae, "RMSE_2024": rmse, "R2_2024": r2}


def fit_and_evaluate_arima(
    config_name: str,
    data: pd.DataFrame,
    hospital_types: List[str],
    infection_cols: Dict[str, str],
    target_total_col: str,
    train_year_start: int = 2017,
    train_year_end: int = 2023,
    test_year: int = 2024,
    arima_order: Tuple[int, int, int] = (1, 1, 0),
) -> Tuple[pd.DataFrame, pd.DataFrame]:

    overall_metrics = _arima_metrics_for_target(
        data=data,
        target_col=target_total_col,
        train_year_start=train_year_start,
        train_year_end=train_year_end,
        test_year=test_year,
        arima_order=arima_order,
    )

    overall_metrics_df = pd.DataFrame([{
        "Config": config_name,
        **overall_metrics
    }])

    facility_rows = []

    for ht in hospital_types:
        tgt_col = infection_cols[ht]

        metrics = _arima_metrics_for_target(
            data=data,
            target_col=tgt_col,
            train_year_start=train_year_start,
            train_year_end=train_year_end,
            test_year=test_year,
            arima_order=arima_order,
        )

        row = {
            "Config": config_name,
            "Hospital_Type": ht,
            **metrics
        }
        facility_rows.append(row)

    per_facility_df = pd.DataFrame(facility_rows)

    return overall_metrics_df, per_facility_df


In [55]:
arima_overall_with_sdoh, arima_facilities_with_sdoh = fit_and_evaluate_arima(
    config_name="arima_with_sdoh",
    data=df_sdoh,
    hospital_types=hospital_types,
    infection_cols=infection_cols,
    target_total_col=target_total_col,
    train_year_start=2017,
    train_year_end=2023,
    test_year=2024,
    arima_order=(1, 1, 0), 
)


c:\Users\Alan\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Alan\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Alan\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Alan\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
c:\Users\Alan\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Lik

### 2.1.4 Random Forrest (RF) Regressor without SDOH

In [56]:
rf_overall_no_sdoh, rf_facilities_no_sdoh = fit_and_evaluate_config(
    config_name="rf_no_sdoh",
    data=df_base,
    feature_cols=feature_cols_rf_basic,
    model_builder=build_rf,
    hospital_types=hospital_types,
    infection_cols=infection_cols,
    target_total_col=target_total_col
)

### 2.1.5 RF Regressor with SDOH

In [57]:
rf_overall_with_sdoh, rf_facilities_with_sdoh = fit_and_evaluate_config(
    config_name="rf_with_sdoh",
    data=df_sdoh,
    feature_cols=feature_cols_rf_sdoh,
    model_builder=build_rf,
    hospital_types=hospital_types,
    infection_cols=infection_cols,
    target_total_col=target_total_col
)


### 2.1.6 RF Regressor with SDOH + lag features

In [58]:
rf_overall_with_sdoh_lag, rf_facilities_with_sdoh_lag = fit_and_evaluate_config(
    config_name="rf_with_sdoh_lag",
    data=df_sdoh_lag,
    feature_cols=feature_cols_rf_lag,
    model_builder=build_rf,
    hospital_types=hospital_types,
    infection_cols=infection_cols,
    target_total_col=target_total_col
)

### 2.1.7 XGBoost Regressor with SDOH + lag features

In [59]:
xgb_overall_with_sdoh_lag, xgb_facilities_with_sdoh_lag = fit_and_evaluate_config(
    config_name="xgb_with_sdoh_lag",
    data=df_sdoh_lag,
    feature_cols=feature_cols_rf_lag,
    model_builder=build_xgb,
    hospital_types=hospital_types,
    infection_cols=infection_cols,
    target_total_col=target_total_col
)

### 2.1.8 Deep Learning (Keras)

In [ ]:

class KerasMRSARegressor(BaseEstimator, RegressorMixin):
    def __init__(
        self,
        epochs: int = 50,
        batch_size: int = 128,
        learning_rate: float = 1e-3,
        verbose: int = 0,
    ):
        self.epochs = epochs
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        self.verbose = verbose

    def _build_model(self, input_dim: int):
        model = keras.Sequential([
            layers.Input(shape=(input_dim,)),
            layers.Dense(128, activation="relu"),
            layers.Dropout(0.2),
            layers.Dense(64, activation="relu"),
            layers.Dropout(0.2),
            layers.Dense(1, activation="linear"),
        ])
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=self.learning_rate),
            loss="mse",
        )
        return model

    def fit(self, X, y):
        X = np.asarray(X, dtype="float32")
        y = np.asarray(y, dtype="float32")

        self.input_dim_ = X.shape[1]
        self.model_ = self._build_model(self.input_dim_)

        self.model_.fit(
            X,
            y,
            validation_split=0.1,
            epochs=self.epochs,
            batch_size=self.batch_size,
            verbose=self.verbose,
            callbacks=[
                keras.callbacks.EarlyStopping(
                    monitor="val_loss",
                    patience=5,
                    restore_best_weights=True,
                )
            ],
        )
        return self

    def predict(self, X):
        X = np.asarray(X, dtype="float32")
        preds = self.model_.predict(X, verbose=0)
        return preds.ravel()


def build_keras():
    return KerasMRSARegressor()

def evaluate_keras_all_facilities(
    data: pd.DataFrame,
    feature_cols: list,
    hospital_types: list,
    infection_cols: dict,
    target_total_col: str,
    train_year_start: int = 2017,
    train_year_end: int = 2023,
    test_year: int = 2024,
) -> Dict[str, float]:

    overall_df, _ = fit_and_evaluate_config(
        config_name="Keras_AllFacilities",
        data=data,
        feature_cols=feature_cols,
        model_builder=build_keras,       
        hospital_types=hospital_types,
        infection_cols=infection_cols,
        target_total_col=target_total_col,
        train_year_start=train_year_start,
        train_year_end=train_year_end,
        test_year=test_year,
    )

    row = overall_df.iloc[0]
    return {
        "MAE": row["MAE_2024"],
        "RMSE": row["RMSE_2024"],
        "R2": row["R2_2024"],
    }


In [ ]:
keras_metrics = evaluate_keras_all_facilities(
    data=df_sdoh_lag,                       
    feature_cols=feature_cols_rf_lag,
    hospital_types=hospital_types,
    infection_cols=infection_cols,
    target_total_col=target_total_col,
)
keras_metrics


{'MAE': 12.725442278598036,
 'RMSE': 35.096695689910476,
 'R2': -0.1505227462706884}

In [ ]:
config_name = "keras_with_sdoh_lag"
mse = keras_metrics['RMSE'] ** 2

# Table data
keras_overall_with_sdoh_lag = pd.DataFrame([{
    "Config": config_name,
    "MAE_2024": keras_metrics["MAE"],
    "MSE_2024": mse,
    "R2_2024": keras_metrics["R2"],
    "RMSE_2024": keras_metrics["RMSE"]
}])


keras_overall_with_sdoh_lag

Config   MAE_2024     MSE_2024   R2_2024  RMSE_2024
0  keras_with_sdoh_lag  12.725442  1231.778048 -0.150523  35.096696

## 2.2 Model Result Compilation

### 2.2.1 All Hospital Results

In [115]:

all_hospital_results = pd.concat([
    lr_overall_no_sdoh,
    lr_overall_with_sdoh,
    arima_overall_with_sdoh,
    rf_overall_no_sdoh,
    rf_overall_with_sdoh,
    rf_overall_with_sdoh_lag,
    xgb_overall_with_sdoh_lag,
    keras_overall_with_sdoh_lag
], axis=0, ignore_index=True)

# all_hospital_results = pd.concat([
#     all_hospital_results,
#     rf_overall_with_sdoh_lag_tuned], axis=0, ignore_index=True)

all_hospital_results = (
    all_hospital_results
        .drop(columns=["MSE_2024"])
        .assign(Hospital_Type="ALL_HOSPITAL_TYPES")
)


all_hospital_results.to_csv("all_hospital_results.csv", index=False)
all_hospital_results

Config    MAE_2024     R2_2024   RMSE_2024       Hospital_Type
0           lr_no_sdoh    4.903544    0.907705    9.940479  ALL_HOSPITAL_TYPES
1         lr_with_sdoh  183.402463 -122.815054  364.087150  ALL_HOSPITAL_TYPES
2      arima_with_sdoh    2.404245    0.981461    4.455119  ALL_HOSPITAL_TYPES
3           rf_no_sdoh    3.746071    0.898629   10.417795  ALL_HOSPITAL_TYPES
4         rf_with_sdoh    6.966393    0.863101   12.106513  ALL_HOSPITAL_TYPES
5     rf_with_sdoh_lag    6.210679    0.902586   10.212424  ALL_HOSPITAL_TYPES
6    xgb_with_sdoh_lag    5.816614    0.846088   12.836725  ALL_HOSPITAL_TYPES
7  keras_with_sdoh_lag   12.725442   -0.150523   35.096696  ALL_HOSPITAL_TYPES

### 2.2.2 Facility-Specific Results

In [ ]:
final_facilities_df = pd.concat([
    lr_facilities_no_sdoh,
    lr_facilities_with_sdoh,
    arima_facilities_with_sdoh,
    rf_facilities_no_sdoh,
    rf_facilities_with_sdoh,
    rf_facilities_with_sdoh_lag,
    xgb_facilities_with_sdoh_lag
], axis=0, ignore_index=True)


# final_facilities_df = pd.concat([
#     final_facilities_df,
#     rf_facilities_with_sdoh_lag_tuned
# ], axis=0, ignore_index=True)
# final_facilities_df



final_facilities_df.to_csv("../../data/report_data/final_facilities_df.csv", index=False)
final_facilities_df

Config                          Hospital_Type  MAE_2024  \
0          lr_no_sdoh                        Critical Access  0.080600   
1          lr_no_sdoh  Free-Standing Rehabilitation Hospital  0.015773   
2          lr_no_sdoh                   Long-Term Acute Care  0.825758   
3          lr_no_sdoh                         Major Teaching  2.898325   
4          lr_no_sdoh                              Pediatric  0.238752   
..                ...                                    ...       ...   
58  xgb_with_sdoh_lag                              Pediatric  0.104199   
59  xgb_with_sdoh_lag                    Rehabilitation Unit  0.061011   
60  xgb_with_sdoh_lag                                     cl  0.777422   
61  xgb_with_sdoh_lag                                     cm  0.739807   
62  xgb_with_sdoh_lag                                     cs  0.652222   

    RMSE_2024   R2_2024  
0    0.235949  0.206419  
1    0.025762  0.000000  
2    1.515028  0.928915  
3    5.697515  0.889630  
4    0.555239  0.709495  
..        ...       ...  
58   0.381569  0.862805  
59   0.151069  0.735908  
60   1.821963  0.899590  
61   2.344792  0.344116  
62   1.080415  0.718910  

[63 rows x 5 columns]

# Section 3: Best-Performing Model (RF_with_SDOH_Lag) Hypertuning

In [ ]:

from sklearn.model_selection import RandomizedSearchCV

df_tune = df_sdoh_lag[(df_sdoh_lag["Year"] >= 2017) & (df_sdoh_lag["Year"] <= 2023)].copy()
X_tune = df_tune[feature_cols_rf_lag]
y_tune = df_tune["Infections_Reported_Total"]

#  preprocessor
preprocessor_tune = make_preprocessor(feature_cols_rf_lag)
X_tune_proc = preprocessor_tune.fit_transform(X_tune)

# searcj
base_rf = RandomForestRegressor(
    random_state=42,
    n_jobs=-1
)

param_dist = {
    "n_estimators": [200, 400, 600, 800],
    "max_depth": [None, 5, 10, 20, 30],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2", 0.3, 0.5, 0.8],
    "bootstrap": [True, False],
}

rf_random = RandomizedSearchCV(
    estimator=base_rf,
    param_distributions=param_dist,
    n_iter=40,                     
    scoring="neg_mean_absolute_error",
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

#tune

rf_random.fit(X_tune_proc, y_tune)

print("Best params for RF_with_SDOH_Lag:")
print(rf_random.best_params_)

best_params_rf_lag = rf_random.best_params_


Fitting 3 folds for each of 40 candidates, totalling 120 fits
Best params for RF_with_SDOH_Lag:
{'n_estimators': 400, 'min_samples_split': 2, 'min_samples_leaf': 4, 'max_features': 0.8, 'max_depth': 10, 'bootstrap': True}


In [ ]:

# 4.  tuned RF for SDOH + Lag

def build_rf_tuned():
    return RandomForestRegressor(
        n_estimators=best_params_rf_lag.get("n_estimators", 500),
        max_depth=best_params_rf_lag.get("max_depth", None),
        min_samples_split=best_params_rf_lag.get("min_samples_split", 2),
        min_samples_leaf=best_params_rf_lag.get("min_samples_leaf", 1),
        max_features=best_params_rf_lag.get("max_features", "sqrt"),
        bootstrap=best_params_rf_lag.get("bootstrap", True),
        random_state=42,
        n_jobs=-1
    )


rf_overall_with_sdoh_lag_tuned, rf_facilities_with_sdoh_lag_tuned = fit_and_evaluate_config(
    config_name="RF_with_SDOH_Lag_TUNED",
    data=df_sdoh_lag,
    feature_cols=feature_cols_rf_lag,
    model_builder=build_rf_tuned,
    hospital_types=hospital_types,
    infection_cols=infection_cols,
    target_total_col=target_total_col
)




In [ ]:

# join back with other dfs

# go back and uncomment above


# Section 4: Best-Perfoming Model Rolling-Horizon CV  + Feature Importance

## 4.1 Best-Perfoming Model Rolling-Horizon CV 

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.pipeline import Pipeline

# buyild best_params_rf_lag
def build_rf_tuned():
    return RandomForestRegressor(
        n_estimators=best_params_rf_lag.get("n_estimators", 400),
        max_depth=best_params_rf_lag.get("max_depth", 10),
        min_samples_split=best_params_rf_lag.get("min_samples_split", 2),
        min_samples_leaf=best_params_rf_lag.get("min_samples_leaf", 4),
        max_features=best_params_rf_lag.get("max_features", 0.8),
        bootstrap=best_params_rf_lag.get("bootstrap", True),
        random_state=42,
        n_jobs=-1
    )

# rolling-origin folds
cv_folds = [
    (2017, 2019, 2020),
    (2017, 2020, 2021),
    (2017, 2021, 2022),
    (2017, 2022, 2023),
    (2017, 2023, 2024),
]

fold_results = []
facility_results = []


# 

for fold_idx, (train_start, train_end, test_year) in enumerate(cv_folds, 1):

    print(f"\n==== FOLD {fold_idx}: Train {train_start}-{train_end}, Test {test_year} ====")

    #  split
    train_df = df_sdoh_lag[(df_sdoh_lag["Year"] >= train_start) & (df_sdoh_lag["Year"] <= train_end)].copy()
    test_df  = df_sdoh_lag[df_sdoh_lag["Year"] == test_year].copy()

    X_train = train_df[feature_cols_rf_lag]
    y_train_total = train_df[target_total_col]
    X_test  = test_df[feature_cols_rf_lag]
    y_test_total = test_df[target_total_col]

    # modeling
    preprocessor = make_preprocessor(feature_cols_rf_lag)
    overall_pipe = Pipeline(steps=[
        ("preprocess", preprocessor),
        ("model", build_rf_tuned())
    ])

    overall_pipe.fit(X_train, y_train_total)
    y_pred_total = overall_pipe.predict(X_test)

    mae = mean_absolute_error(y_test_total, y_pred_total)
    rmse = np.sqrt(mean_squared_error(y_test_total, y_pred_total))
    r2 = r2_score(y_test_total, y_pred_total)

    fold_results.append({
        "Fold": fold_idx,
        "Train_Years": f"{train_start}-{train_end}",
        "Test_Year": test_year,
        "MAE": mae,
        "RMSE": rmse,
        "R2": r2
    })

    print(f"Total infections → MAE={mae:.4f}, RMSE={rmse:.4f}, R²={r2:.44f}")

    #  facility-specific models
    for ht in hospital_types:
        tgt_col = infection_cols[ht]

        y_train_fac = train_df[tgt_col]
        y_test_fac  = test_df[tgt_col]

        fac_pipe = Pipeline(steps=[
            ("preprocess", make_preprocessor(feature_cols_rf_lag)),
            ("model", build_rf_tuned())
        ])

        fac_pipe.fit(X_train, y_train_fac)
        y_pred_fac = fac_pipe.predict(X_test)

        mae_ft = mean_absolute_error(y_test_fac, y_pred_fac)
        rmse_ft = np.sqrt(mean_squared_error(y_test_fac, y_pred_fac))

        facility_results.append({
            "Fold": fold_idx,
            "Train_Years": f"{train_start}-{train_end}",
            "Test_Year": test_year,
            "Hospital_Type": ht,
            "MAE": mae_ft,
            "RMSE": rmse_ft
        })

fold_results_df = pd.DataFrame(fold_results)
facility_results_df = pd.DataFrame(facility_results)

fold_results_df





==== FOLD 1: Train 2017-2019, Test 2020 ====
Total infections → MAE=5.2827, RMSE=18.7722, R²=0.72186364151303261671444033709121868014335632

==== FOLD 2: Train 2017-2020, Test 2021 ====
Total infections → MAE=6.8613, RMSE=22.9329, R²=0.74167264455367809539154677622718736529350281

==== FOLD 3: Train 2017-2021, Test 2022 ====
Total infections → MAE=5.6323, RMSE=17.5797, R²=0.84416248717410602431243660248583182692527771

==== FOLD 4: Train 2017-2022, Test 2023 ====
Total infections → MAE=4.4620, RMSE=9.4164, R²=0.92967530301591116970172379296855069696903229

==== FOLD 5: Train 2017-2023, Test 2024 ====
Total infections → MAE=5.3583, RMSE=9.0236, R²=0.92394671498672420462128229701193049550056458


(   Fold Train_Years  Test_Year       MAE       RMSE        R2
 0     1   2017-2019       2020  5.282670  18.772202  0.721864
 1     2   2017-2020       2021  6.861329  22.932886  0.741673
 2     3   2017-2021       2022  5.632284  17.579682  0.844162
 3     4   2017-2022       2023  4.462040   9.416424  0.929675
 4     5   2017-2023       2024  5.358262   9.023554  0.923947,
     Fold Train_Years  Test_Year                          Hospital_Type  \
 0      1   2017-2019       2020                        Critical Access   
 1      1   2017-2019       2020  Free-Standing Rehabilitation Hospital   
 2      1   2017-2019       2020                   Long-Term Acute Care   
 3      1   2017-2019       2020                         Major Teaching   
 4      1   2017-2019       2020                              Pediatric   
 5      1   2017-2019       2020                    Rehabilitation Unit   
 6      1   2017-2019       2020                                     cl   
 7      1   2017-2019

In [103]:
facility_results_df

Fold Train_Years  Test_Year                          Hospital_Type  \
0      1   2017-2019       2020                        Critical Access   
1      1   2017-2019       2020  Free-Standing Rehabilitation Hospital   
2      1   2017-2019       2020                   Long-Term Acute Care   
3      1   2017-2019       2020                         Major Teaching   
4      1   2017-2019       2020                              Pediatric   
5      1   2017-2019       2020                    Rehabilitation Unit   
6      1   2017-2019       2020                                     cl   
7      1   2017-2019       2020                                     cm   
8      1   2017-2019       2020                                     cs   
9      2   2017-2020       2021                        Critical Access   
10     2   2017-2020       2021  Free-Standing Rehabilitation Hospital   
11     2   2017-2020       2021                   Long-Term Acute Care   
12     2   2017-2020       2021                         Major Teaching   
13     2   2017-2020       2021                              Pediatric   
14     2   2017-2020       2021                    Rehabilitation Unit   
15     2   2017-2020       2021                                     cl   
16     2   2017-2020       2021                                     cm   
17     2   2017-2020       2021                                     cs   
18     3   2017-2021       2022                        Critical Access   
19     3   2017-2021       2022  Free-Standing Rehabilitation Hospital   
20     3   2017-2021       2022                   Long-Term Acute Care   
21     3   2017-2021       2022                         Major Teaching   
22     3   2017-2021       2022                              Pediatric   
23     3   2017-2021       2022                    Rehabilitation Unit   
24     3   2017-2021       2022                                     cl   
25     3   2017-2021       2022                                     cm   
26     3   2017-2021       2022                                     cs   
27     4   2017-2022       2023                        Critical Access   
28     4   2017-2022       2023  Free-Standing Rehabilitation Hospital   
29     4   2017-2022       2023                   Long-Term Acute Care   
30     4   2017-2022       2023                         Major Teaching   
31     4   2017-2022       2023                              Pediatric   
32     4   2017-2022       2023                    Rehabilitation Unit   
33     4   2017-2022       2023                                     cl   
34     4   2017-2022       2023                                     cm   
35     4   2017-2022       2023                                     cs   
36     5   2017-2023       2024                        Critical Access   
37     5   2017-2023       2024  Free-Standing Rehabilitation Hospital   
38     5   2017-2023       2024                   Long-Term Acute Care   
39     5   2017-2023       2024                         Major Teaching   
40     5   2017-2023       2024                              Pediatric   
41     5   2017-2023       2024                    Rehabilitation Unit   
42     5   2017-2023       2024                                     cl   
43     5   2017-2023       2024                                     cm   
44     5   2017-2023       2024                                     cs   

         MAE       RMSE  
0   0.023644   0.056329  
1   0.010542   0.027166  
2   0.916678   2.969965  
3   2.815480   8.468387  
4   0.086728   0.271471  
5   0.145384   0.352456  
6   1.905001   6.440348  
7   1.173027   2.467720  
8   0.580903   1.732334  
9   0.103436   0.287078  
10  0.007002   0.023085  
11  0.641231   2.040048  
12  3.557773  12.318732  
13  0.226556   0.895703  
14  0.105628   0.219737  
15  1.339287   4.009275  
16  1.316976   3.136247  
17  0.883394   2.037691  
18  0.067930   0.257416  
19  0.005143   0.023683  
20  0.837579   2.576433  
21  2.897667  10.583236  
2

# 4.2 Feature Importance

In [ ]:

X_rf_lag = df_sdoh_lag[feature_cols_rf_lag].copy()
y_rf_lag = df_sdoh_lag[target_total_col].copy()

categorical_cols = X_rf_lag.select_dtypes(include=["object", "category"]).columns.tolist()
numeric_cols = [c for c in X_rf_lag.columns if c not in categorical_cols]

# preprocessing + model pipeline
preprocessor_rf_lag = ColumnTransformer(
    transformers=[
        ("num", "passthrough", numeric_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), categorical_cols),
    ]
)

rf_tuned_for_importance = build_rf_tuned()

rf_pipeline_for_importance = Pipeline(
    steps=[
        ("preprocessor", preprocessor_rf_lag),
        ("model", rf_tuned_for_importance),
    ]
)

# fit
rf_pipeline_for_importance.fit(X_rf_lag, y_rf_lag)

# feature names 
ohe = rf_pipeline_for_importance.named_steps["preprocessor"].named_transformers_["cat"]
cat_feature_names = ohe.get_feature_names_out(categorical_cols)
all_feature_names = numeric_cols + list(cat_feature_names)

#feature importances 
importances = rf_pipeline_for_importance.named_steps["model"].feature_importances_

rf_feature_importance_df = (
    pd.DataFrame({
        "Config": "RF_with_SDOH_Lag_TUNED",
        "Feature": all_feature_names,
        "Importance": importances
    })
    .sort_values("Importance", ascending=False)
    .reset_index(drop=True)
)

rf_feature_importance_df.head(10)


Config                                         Feature  \
0  RF_with_SDOH_Lag_TUNED                                     Labor Force   
1  RF_with_SDOH_Lag_TUNED                    patient_days__Major Teaching   
2  RF_with_SDOH_Lag_TUNED                                patient_days__cm   
3  RF_with_SDOH_Lag_TUNED                               ranked_Population   
4  RF_with_SDOH_Lag_TUNED                         patient_days__Pediatric   
5  RF_with_SDOH_Lag_TUNED  infections_reported__Long-Term Acute Care_lag1   
6  RF_with_SDOH_Lag_TUNED        infections_reported__Major Teaching_lag1   
7  RF_with_SDOH_Lag_TUNED                                patient_days__cs   
8  RF_with_SDOH_Lag_TUNED               patient_days__Rehabilitation Unit   
9  RF_with_SDOH_Lag_TUNED                              County_Los Angeles   

   Importance  
0    0.236781  
1    0.099352  
2    0.077423  
3    0.065827  
4    0.053197  
5    0.052217  
6    0.049634  
7    0.048618  
8    0.047482  
9    0.047251

| Rank   | Feature Name                                       | Importance   | Meaning (Updated)                                                                             |
| ------ | -------------------------------------------------- | ------------ | --------------------------------------------------------------------------------------------- |
| **1**  | **Labor Force**                                    | **0.236781** | County-level labor force size (major socioeconomic SDOH variable).                            |
| **2**  | **patient_days__Major Teaching**                   | **0.099352** | Total patient-days in **Major Teaching** hospitals (large academic centers with high acuity). |
| **3**  | **patient_days__cm**                               | **0.077423** | Patient-days in **Community Medium** hospitals (125–250 beds).                                |
| **4**  | **ranked_Population**                              | **0.065827** | Ranked/percentile-transformed county population (SDOH scale indicator).                       |
| **5**  | **patient_days__Pediatric**                        | **0.053197** | Patient-days in **Pediatric** hospitals/units.                                                |
| **6**  | **infections_reported__Long-Term Acute Care_lag1** | **0.052217** | Prior-period infections in **Long-Term Acute Care** hospitals (lag-1 outcome signal).         |
| **7**  | **infections_reported__Major Teaching_lag1**       | **0.049634** | Prior-period infections in **Major Teaching** hospitals (lag-1).                              |
| **8**  | **patient_days__cs**                               | **0.048618** | Patient-days in **Community Small** hospitals (<125 beds).                                    |
| **9**  | **patient_days__Rehabilitation Unit**              | **0.047482** | Patient-days in **Rehabilitation Units** (post-acute specialty care).                         |
| **10** | **County_Los Angeles**                             | **0.047251** | Dummy indicator for **Los Angeles County** (large fixed geographic effect).                   |




### **1. SDOH signals dominate the model**

- **Labor Force** (0.2368) is the strongest overall predictor of infection burden.  
- **ranked_Population** (0.0658) also contributes significantly.  
-  **community-level socioeconomic factors heavily influence infection levels**.

---

### **2. Patient volume (exposure) is the second major driver**

High-importance hospital exposure variables:

- `patient_days__Major Teaching`  
- `patient_days__cm` — *Community Medium hospitals (125–250 beds)*  
- `patient_days__cs` — *Community Small hospitals (<125 beds)*  
- `patient_days__Pediatric`  
- `patient_days__Rehabilitation Unit`  

These show that **facility size, complexity, and specialty mix are major contributors to infection burden**.

---

### **3. Lagged infections meaningfully improve prediction**

- `infections_reported__Long-Term Acute Care_lag1`  
- `infections_reported__Major Teaching_lag1`  

 Past infection levels provide **temporal continuity**, improving predictive accuracy.

---

### **4. Geography matters**

- `County_Los Angeles` has non-trivial importance (0.0473)

Suggests **regional factors uniquely affect infection behavior** beyond SDOH or hospital characteristics.

---

## Importance Concentration

The sum of the top 10 feature importances is:

\[
0.2368 + 0.0994 + 0.0774 + 0.0658 + 0.0532 + 0.0522 + 0.0496 + 0.0486 + 0.0475 + 0.0473 \approx 0.78
\]

 **The top 10 features account for ~78% of the total model importance.**

---

##  Category Mapping (for clarity)

- **SDOH Variables**
  - Labor Force  
  - ranked_Population  

- **Hospital Exposure Variables**
  - patient_days__Major Teaching  
  - patient_days__cm — Community Medium (125–250 beds)  
  - patient_days__cs — Community Small (<125 beds)  
  - patient_days__Pediatric  
  - patient_days__Rehabilitation Unit  

- **Lagged Infection Variables**
  - infections_reported__Long-Term Acute Care_lag1  
  - infections_reported__Major Teaching_lag1  

- **Geographic Indicators**
  - County_Los Angeles  

